# Building an image retrieval system with deep features


# Fire up GraphLab Create
(See [Getting Started with SFrames](../Week%201/Getting%20Started%20with%20SFrames.ipynb) for setup instructions)

In [1]:
import graphlab

In [2]:
# Limit number of worker processes. This preserves system memory, which prevents hosted notebooks from crashing.
graphlab.set_runtime_config('GRAPHLAB_DEFAULT_NUM_PYLAMBDA_WORKERS', 4)

[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: C:\Users\STRIKE~1\AppData\Local\Temp\graphlab_server_1538180767.log.0


This non-commercial license of GraphLab Create for academic use is assigned to hongng234@gmail.com and will expire on September 06, 2019.


# Load the CIFAR-10 dataset

We will use a popular benchmark dataset in computer vision called CIFAR-10.  

(We've reduced the data to just 4 categories = {'cat','bird','automobile','dog'}.)

This dataset is already split into a training set and test set. In this simple retrieval example, there is no notion of "testing", so we will only use the training data.

In [3]:
image_train = graphlab.SFrame('image_train_data.csv')

Finished parsing file C:\Users\StrikeWade\Desktop\DLML\Machine-Learning-Courses\Coursera_Machine_Learning\1_Machine_Learning_Foundations_A_Case_Study_Approach\Week 6 - Deep Learning\image_train_data.csv

Parsing completed. Parsed 100 lines in 0.258948 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[long,str,str,array,array]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Read 1943 lines. Lines per second: 3151.31

Finished parsing file C:\Users\StrikeWade\Desktop\DLML\Machine-Learning-Courses\Coursera_Machine_Learning\1_Machine_Learning_Foundations_A_Case_Study_Approach\Week 6 - Deep Learning\image_train_data.csv

Parsing completed. Parsed 2005 lines in 0.616569 secs.

# Computing deep features for our images

The two lines below allow us to compute deep features.  This computation takes a little while, so we have already computed them and saved the results as a column in the data you loaded. 

(Note that if you would like to compute such deep features and have a GPU on your machine, you should use the GPU enabled GraphLab Create, which will be significantly faster for this task.)

In [4]:
# deep_learning_model = graphlab.load_model('http://s3.amazonaws.com/GraphLab-Datasets/deeplearning/imagenet_model_iter45')
# image_train['deep_features'] = deep_learning_model.extract_features(image_train)

In [5]:
image_train.head()

id,image,label,deep_features,image_array
24,Height: 32 Width: 32,bird,"[0.242872, 1.09545, 0.0,0.39363, 0.0, 0.0, ...","[73.0, 77.0, 58.0, 71.0,68.0, 50.0, 77.0, 69.0, ..."
33,Height: 32 Width: 32,cat,"[0.525088, 0.0, 0.0, 0.0,0.0, 0.0, 9.94829, 0.0, ...","[7.0, 5.0, 8.0, 7.0, 5.0,8.0, 5.0, 4.0, 6.0, 7.0, ..."
36,Height: 32 Width: 32,cat,"[0.566016, 0.0, 0.0, 0.0,0.0, 0.0, 9.9972, 0.0, ...","[169.0, 122.0, 65.0,131.0, 108.0, 75.0, ..."
70,Height: 32 Width: 32,dog,"[1.1298, 0.0, 0.0,0.778194, 0.0, 0.758051, ...","[154.0, 179.0, 152.0,159.0, 183.0, 157.0, ..."
90,Height: 32 Width: 32,bird,"[1.71787, 0.0, 0.0, 0.0,0.0, 0.0, 9.33936, 0.0, ...","[216.0, 195.0, 180.0,201.0, 178.0, 160.0, ..."
97,Height: 32 Width: 32,automobile,"[1.57819, 0.0, 0.0, 0.0,0.0, 0.0, 9.00632, 0.0, ...","[33.0, 44.0, 27.0, 29.0,44.0, 31.0, 32.0, 45.0, ..."
107,Height: 32 Width: 32,dog,"[0.0, 0.0, 0.220678, 0.0,0.0, 0.0, 8.58053, ...","[97.0, 51.0, 31.0, 104.0,58.0, 38.0, 107.0, 61.0, ..."
121,Height: 32 Width: 32,bird,"[0.0, 0.237535, 0.0, 0.0,0.0, 0.0, 9.9908, 0.0, ...","[93.0, 96.0, 88.0, 102.0,106.0, 97.0, 117.0, ..."
136,Height: 32 Width: 32,automobile,"[0.0, 0.0, 0.0, 0.0, 0.0,0.0, 7.57379, 0.0, 0.0, ...","[35.0, 59.0, 53.0, 36.0,56.0, 56.0, 42.0, 62.0, ..."
138,Height: 32 Width: 32,bird,"[0.658936, 0.0, 0.0, 0.0,0.0, 0.0, 9.93748, 0.0, ...","[205.0, 193.0, 195.0,200.0, 187.0, 193.0, ..."


# Train a nearest-neighbors model for retrieving images using deep features

We will now build a simple image retrieval system that finds the nearest neighbors for any image.

In [6]:
knn_model = graphlab.nearest_neighbors.create(
    image_train,
    features=['deep_features'],
    label='id')

Starting brute force nearest neighbors model training.

# Use image retrieval model with deep features to find similar images

Let's find similar images to this cat picture.

In [7]:
graphlab.canvas.set_target('ipynb')
cat = image_train[18:19]
cat['image'].show()

In [8]:
knn_model.query(cat)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.0498753   | 22.992ms     |

| Done         |         | 100         | 114.96ms     |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,384,0.0,1
0,6910,36.940312418,2
0,39777,38.4634892031,3
0,36870,39.7559665724,4
0,41734,39.7865972971,5


We are going to create a simple function to view the nearest neighbors to save typing:

In [9]:
def get_images_from_ids(query_result):
    return image_train.filter_by(query_result['reference_label'],'id')

In [10]:
cat_neighbors = get_images_from_ids(knn_model.query(cat))

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.0498753   | 24.993ms     |

| Done         |         | 100         | 118.96ms     |

+--------------+---------+-------------+--------------+

In [11]:
cat_neighbors['image'].show()

Very cool results showing similar cats.

## Finding similar images to a car

In [12]:
car = image_train[8:9]
car['image'].show()

In [13]:
get_images_from_ids(knn_model.query(car))['image'].show()

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.0498753   | 26.992ms     |

| Done         |         | 100         | 104.967ms    |

+--------------+---------+-------------+--------------+

# Just for fun, let's create a lambda to find and show nearest neighbor images

In [14]:
show_neighbors = lambda i: get_images_from_ids(knn_model.query(image_train[i:i+1]))['image'].show()

In [15]:
show_neighbors(8)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.0498753   | 26.982ms     |

| Done         |         | 100         | 106.953ms    |

+--------------+---------+-------------+--------------+

In [16]:
show_neighbors(26)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.0498753   | 22.996ms     |

| Done         |         | 100         | 97.968ms     |

+--------------+---------+-------------+--------------+